In [1]:
"""
linear_regression_scratch.py
----------------------------
A scikit-learn-like Linear Regression implemented from scratch using NumPy.

Features
- API similar to sklearn: fit(X, y), predict(X), score(X, y), get_params(), set_params()
- Solvers: "normal" (closed-form) or "gd" (gradient descent)
- Optional L2 regularization (ridge)
- fit_intercept handling (intercept is never regularized)
- Optional standardization for GD stability
- Metrics: mse, mae, r2_score
"""

from __future__ import annotations
from dataclasses import dataclass, field
from typing import Optional, Dict, Any, Tuple
import numpy as np


In [2]:
# ---------- Metrics ----------
def mse(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    y_true = np.asarray(y_true).reshape(-1)
    y_pred = np.asarray(y_pred).reshape(-1)
    return float(np.mean((y_true - y_pred) ** 2))


def mae(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    y_true = np.asarray(y_true).reshape(-1)
    y_pred = np.asarray(y_pred).reshape(-1)
    return float(np.mean(np.abs(y_true - y_pred)))


def r2_score(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    y_true = np.asarray(y_true).reshape(-1)
    y_pred = np.asarray(y_pred).reshape(-1)
    ss_res = float(np.sum((y_true - y_pred) ** 2))
    ss_tot = float(np.sum((y_true - np.mean(y_true)) ** 2))
    return 1.0 - ss_res / ss_tot if ss_tot != 0 else 0.0


In [3]:
# ---------- Utilities ----------
@dataclass
class _StandardScaler:
    """Minimal standard scaler for GD stability."""
    mean_: Optional[np.ndarray] = field(default=None, init=False)
    std_: Optional[np.ndarray]  = field(default=None, init=False)
    eps: float = 1e-12

    def fit(self, X: np.ndarray) -> "_StandardScaler":
        X = np.asarray(X, dtype=float)
        self.mean_ = X.mean(axis=0)
        self.std_  = X.std(axis=0)
        # Prevent divide-by-zero for constant columns
        self.std_[self.std_ < self.eps] = 1.0
        return self

    def transform(self, X: np.ndarray) -> np.ndarray:
        if self.mean_ is None or self.std_ is None:
            raise RuntimeError("Scaler not fitted.")
        X = np.asarray(X, dtype=float)
        return (X - self.mean_) / self.std_

    def fit_transform(self, X: np.ndarray) -> np.ndarray:
        return self.fit(X).transform(X)


def _check_X_y(X: np.ndarray, y: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    X = np.asarray(X, dtype=float)
    y = np.asarray(y, dtype=float).reshape(-1)
    if X.ndim == 1:
        X = X.reshape(-1, 1)
    if X.shape[0] != y.shape[0]:
        raise ValueError(f"X and y have incompatible shapes: {X.shape} vs {y.shape}")
    return X, y


In [8]:
# ---------- Model ----------
class LinearRegression:
    """
    Linear Regression (from scratch) with a scikit-learn-like interface.

    Parameters
    ----------
    fit_intercept : bool, default=True
        Whether to fit the intercept term.
    solver : {"normal", "gd"}, default="normal"
        "normal": closed-form solution (optionally ridge via l2 > 0).
        "gd": gradient descent (useful for large data or to mimic iterative solvers).
    l2 : float, default=0.0
        L2 regularization strength (ridge). Intercept is never regularized.
    learning_rate : float, default=1e-2
        Step size for gradient descent (used only if solver="gd").
    n_iters : int, default=10_000
        Max iterations for gradient descent.
    tol : float, default=1e-8
        Early-stopping tolerance for GD based on loss improvement.
    standardize : bool, default=False
        If True and solver="gd", standardizes features internally for stability.
        Predictions are automatically unscaled; users never see scaled features.
    random_state : Optional[int], default=None
        Seed for GD weight initialization.
    verbose : bool, default=False
        If True, prints convergence info for GD.

    Attributes
    ----------
    coef_ : np.ndarray of shape (n_features,)
        Fitted coefficients (weights) for features.
    intercept_ : float
        Fitted intercept.
    loss_history_ : list[float]
        Loss values per GD iteration (for solver="gd").
    """

    def __init__(
        self,
        fit_intercept: bool = True,
        solver: str = "normal",
        l2: float = 0.0,
        learning_rate: float = 1e-2,
        n_iters: int = 10_000,
        tol: float = 1e-8,
        standardize: bool = False,
        random_state: Optional[int] = None,
        verbose: bool = False,
    ):
        if solver not in {"normal", "gd"}:
            raise ValueError("solver must be 'normal' or 'gd'")
        self.fit_intercept = fit_intercept
        self.solver = solver
        self.l2 = float(l2)
        self.learning_rate = float(learning_rate)
        self.n_iters = int(n_iters)
        self.tol = float(tol)
        self.standardize = bool(standardize)
        self.random_state = random_state
        self.verbose = verbose

        # Learned params
        self.coef_: Optional[np.ndarray] = None
        self.intercept_: float = 0.0
        self.loss_history_: list[float] = []

        # Internal scaler for GD
        self._scaler: Optional[_StandardScaler] = None

    # ---- public API ----
    def fit(self, X: np.ndarray, y: np.ndarray) -> "LinearRegression":
        X, y = _check_X_y(X, y)
        n_samples, n_features = X.shape

        if self.solver == "normal":
            b, w = self._fit_normal_equation(X, y)
            self.intercept_ = b
            self.coef_ = w
            self.loss_history_.clear()
            return self

        # solver == "gd"
        Xs = X
        self._scaler = None
        if self.standardize:
            self._scaler = _StandardScaler().fit(X)
            Xs = self._scaler.transform(X)

        # Build design matrix
        if self.fit_intercept:
            Xb = np.c_[np.ones((n_samples, 1)), Xs]  # [1, x1, ..., xp]
        else:
            Xb = Xs

        # Initialize weights
        rng = np.random.default_rng(self.random_state)
        w = rng.normal(loc=0.0, scale=0.01, size=(Xb.shape[1],))

        self.loss_history_.clear()
        prev_loss = np.inf

        for it in range(self.n_iters):
            y_pred = Xb @ w
            resid = y_pred - y
            loss = float(np.mean(resid ** 2))
            # L2 regularization (do not penalize intercept)
            if self.l2 > 0:
                if self.fit_intercept:
                    loss += self.l2 * float(np.sum(w[1:] ** 2)) / n_samples
                else:
                    loss += self.l2 * float(np.sum(w ** 2)) / n_samples

            self.loss_history_.append(loss)

            # Gradient
            grad = (2.0 / n_samples) * (Xb.T @ resid)
            if self.l2 > 0:
                if self.fit_intercept:
                    grad[1:] += (2.0 * self.l2 / n_samples) * w[1:]
                else:
                    grad += (2.0 * self.l2 / n_samples) * w

            # Update
            w -= self.learning_rate * grad

            # Early stopping
            if abs(prev_loss - loss) < self.tol:
                if self.verbose:
                    print(f"[GD] Converged at iter={it}, loss={loss:.6f}")
                break
            prev_loss = loss

        if self.fit_intercept:
            self.intercept_ = float(w[0])
            self.coef_ = w[1:].copy()
        else:
            self.intercept_ = 0.0
            self.coef_ = w.copy()

        return self

    def predict(self, X: np.ndarray) -> np.ndarray:
        if self.coef_ is None:
            raise RuntimeError("Model not fitted. Call fit(...) first.")
        X = np.asarray(X, dtype=float)
        if X.ndim == 1:
            X = X.reshape(-1, 1)

        Xs = X
        if self.solver == "gd" and self.standardize and self._scaler is not None:
            Xs = self._scaler.transform(X)

        return (self.intercept_ + Xs @ self.coef_).astype(float)

    def score(self, X: np.ndarray, y: np.ndarray) -> float:
        """R² score."""
        return r2_score(y, self.predict(X))

    # scikit-learn style helpers
    def get_params(self, deep: bool = True) -> Dict[str, Any]:
        return {
            "fit_intercept": self.fit_intercept,
            "solver": self.solver,
            "l2": self.l2,
            "learning_rate": self.learning_rate,
            "n_iters": self.n_iters,
            "tol": self.tol,
            "standardize": self.standardize,
            "random_state": self.random_state,
            "verbose": self.verbose,
        }

    def set_params(self, **params: Any) -> "LinearRegression":
        for k, v in params.items():
            if not hasattr(self, k):
                raise ValueError(f"Unknown parameter: {k}")
            setattr(self, k, v)
        return self
      # ---- internal: normal equation ----
    def _fit_normal_equation(self, X: np.ndarray, y: np.ndarray) -> Tuple[float, np.ndarray]:
        """
        Closed-form solution:
            w = (X^T X + λI)^(-1) X^T y
        If fit_intercept=True, the bias column is included but NOT regularized.
        """
        n_samples, n_features = X.shape
        if self.fit_intercept:
            Xb = np.c_[np.ones((n_samples, 1)), X]
            n_params = n_features + 1
        else:
            Xb = X
            n_params = n_features

        I = np.eye(n_params)
        if self.fit_intercept:
            I[0, 0] = 0.0  # don't regularize intercept

        reg = self.l2 * I
        XtX = Xb.T @ Xb
        Xty = Xb.T @ y

        try:
            w = np.linalg.solve(XtX + reg, Xty)
        except np.linalg.LinAlgError:
            # fallback to pseudo-inverse if singular
            w = np.linalg.pinv(XtX + reg) @ Xty

        if self.fit_intercept:
            intercept = float(w[0])
            coef = w[1:].astype(float)
        else:
            intercept = 0.0
            coef = w.astype(float)

        return intercept, coef



In [9]:
# ---------- Example usage ----------
if __name__ == "__main__":
    # Small demo dataset: rooms, area -> price
    data = {
        "h1": [3, 120, 30000],
        "h2": [2, 105, 21900],
        "h3": [5, 200, 61200],
        "h4": [4, 185, 49800],
        "h5": [2, 100, 22700],
        "h6": [3, 118, 33400],
        "h7": [4, 190, 51600],
    }
    X = np.array([v[:2] for v in data.values()], dtype=float)
    y = np.array([v[2] for v in data.values()], dtype=float)

    # Normal equation (exact/closed-form)
    model_ne = LinearRegression(solver="normal", fit_intercept=True, l2=0.0)
    model_ne.fit(X, y)
    print("=== Normal Equation ===")
    print("Intercept:", model_ne.intercept_)
    print("Coefficients:", model_ne.coef_)
    print("R²:", model_ne.score(X, y))

    # Gradient descent (iterative) with standardization
    model_gd = LinearRegression(
        solver="gd",
        fit_intercept=True,
        l2=0.0,
        learning_rate=0.05,
        n_iters=50_000,
        tol=1e-10,
        standardize=True,
        random_state=0,
        verbose=False,
    )
    model_gd.fit(X, y)
    print("\n=== Gradient Descent ===")
    print("Intercept:", model_gd.intercept_)
    print("Coefficients:", model_gd.coef_)
    print("R²:", model_gd.score(X, y))

    # Predictions
    sample = np.array([[3, 150], [5, 210]], dtype=float)
    print("\nPredictions (NE):", model_ne.predict(sample))
    print("Predictions (GD):", model_gd.predict(sample))

=== Normal Equation ===
Intercept: -9999.388472194058
Coefficients: [7569.87296439  163.54483411]
R²: 0.9930264482849642

=== Gradient Descent ===
Intercept: 38657.142857142826
Coefficients: [7798.16000786 6665.5360461 ]
R²: 0.9930264482849632

Predictions (NE): [37241.95553754 62194.39151294]
Predictions (GD): [37241.95610448 62194.39139681]
